In [27]:
"""
Program expects Data/A/100/something.pdf structure
Create a "splitted" folder for the output and then run the program
Splits the pages in half if the longest line is less than 60 characters a.k.a
there are 2 columns of text
"""
import pdfminer
import pdfrw
import alive_progress
import statistics

import sys
import os
from os.path import exists
import shutil
from pdfminer.high_level import extract_text
 
directory = "/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/all_cla_pdfs/"
from pdfrw import PdfReader, PdfWriter, PageMerge

#directory to put the pdfs that have been split
vert_split_dir= '/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_pdfs2'
standard_pdfs = '/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/standard_pdfs'
outside_pdfs = '/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/outside_pdfs'
#os.mkdir(vert_split_dir)

#function to split pdf pages in half
def splitpage(src):
    """Split a page into two (left and right)"""
    # Yield a result for each half of the page
    for x_pos in (0, 0.5):
        yield PageMerge().add(src, viewrect=(x_pos, 0, 0.5, 1)).render()
 
#count of split pdfs and dictionary with totals after each folder is processed
split = 0
unsplit = 0
split_dict = {}
unsplit_dict = {}
#get list of folders from relevant directory that has pdfs needing to be split 
jc_numbers= os.listdir(directory)
print(jc_numbers)
for dir in jc_numbers:
    print(dir)
    for document in os.listdir(f"{directory}/{dir}"):
        if exists(f"{directory}/{dir}/{document}"):
            text = extract_text(f"{directory}/{dir}/{document}")
            text = text.splitlines()
            x=[]
            for i in text:
                if len(i)>40:
                    x.append(len(i))
            try: 
                avg_len_line = round(statistics.mean(x), 2)
            except:
                 avg_len_line = 100
            try:
                med_len_line = statistics.median(x)
            except:
                med_len_line = 100
                print ('Error on file:',document,'\n')
            try:
                max_len_line = max(x)
            except:
                print('all max < 40')
                if os.path.exists(f"{outside_pdfs}/{dir}"):
                    shutil.copy(f"{directory}/{dir}/{document}", f"{outside_pdfs}/{dir}/{document}")
                    print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
                    print('pdf outside' + str(document))    
                else : 
                    os.mkdir(f"{outside_pdfs}/{dir}")
                    shutil.copy(f"{directory}/{dir}/{document}", f"{outside_pdfs}/{dir}/{document}")
                    print('pdf outside' + str(document))

            writer = PdfWriter()
            if max_len_line < 67 or (max_len_line > 67 and avg_len_line < 55 and med_len_line < 55) :
                
                page_number = 0
                for page in PdfReader(f"{directory}/{dir}/{document}").pages:
                    writer.addpages(splitpage(page))
                
                if os.path.exists(f"{vert_split_dir}/{dir}"):
                    writer.write(f"{vert_split_dir}/{dir}/{document}")
                    
                    split += 1
                else:
                    os.mkdir(f"{vert_split_dir}/{dir}")
                    writer.write(f"{vert_split_dir}/{dir}/{document}")
                    print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
                    split += 1
                    
            elif max_len_line > 67 and avg_len_line > 55 and med_len_line > 55:
                if os.path.exists(f"{standard_pdfs}/{dir}"):
                    shutil.copy(f"{directory}/{dir}/{document}", f"{standard_pdfs}/{dir}/{document}")
                    
                    unsplit += 1
                else:
                    os.mkdir(f"{standard_pdfs}/{dir}")
                    shutil.copy(f"{directory}/{dir}/{document}", f"{standard_pdfs}/{dir}/{document}")
                    print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
                    unsplit += 1 
           
            # else:
            #     if os.path.exists(f"{outside_pdfs}/{dir}"):
            #         shutil.copy(f"{directory}/{dir}/{document}", f"{outside_pdfs}/{dir}/{document}")
            #         print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
            #         print('pdf outside' + str(document))    
            #     else : 
            #         os.mkdir(f"{outside_pdfs}/{dir}")
            #         shutil.copy(f"{directory}/{dir}/{document}", f"{outside_pdfs}/{dir}/{document}")
                    
            #         print('pdf outside' + str(document))
        
        else:
            print('error at directory: ' + str(dir))
            
    split_dict[dir] = split 
    unsplit_dict[dir] = unsplit          
    print(f"Split pdfs: {split}")
    print(split_dict)
    print(f"regular pdfs: {unsplit}")
    print(unsplit_dict)

['210', '110', '121', '149.02', '202.01', '202', '200', '112', '102.08', '106.01', '315.02', '105', '324', '148', '109', '115', '315.01', '315.03', '322.01', '309', '319.01', '224', '100', '129', '219', '102.07', '326', '140.01', '125.02', '140', '301.05', '140.03', '339.03', '209', '217', '207', '116', '211', '145', '102.03', '336', '104', '203', '314', '332', '316', '124', '225', '144', '312', '328.02', '113', '225.02', '215', '301.01', '143', '313', '133', '320', '310', '325', '102.05', '146', '220', '331', '337', '303.01', '317', '339.02', '303.03', '102.06', '329.01', '323', '120.02', '120', '340', '214', '327.02', '140.05', '328.01', '225.01', '102.01', '125.01', '339.01', '223', '319.02', '149.01', '102.02', '113.04', '322', '226', '125', '152.01', '140.02', '139', '117', '107', '301', '149.04', '102.04', '300', '127', '321', '152', '329.02', '302', '333', '201', '304', '341', '222', '111', '318.02', '130', '142', '118', '126', '152.02', '140.04', '216', '318.01', '114', '327.03

In [ ]:
#if running functions after eachother this path is not needed, can be split_dir
#but otherwise make this path 
path = "/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_pdfs"

#join pdf half pages into full pdfs, in this case by language
pc_list = os.listdir(path)
for pc_folder in os.listdir(path):
    for document in os.listdir(f"{path}/{pc_folder}"):
        writer_NL = PdfWriter()
        writer_FR = PdfWriter()
        page_number = 0
        for page in PdfReader(f"{path}/{pc_folder}/{document}").pages:
            page_number += 1
            if page_number%2 == 1:
                writer_NL.addpage(page)
            else:
                writer_FR.addpage(page)
        try:
            os.mkdir('./data/split_nl/' + str(document)[0:3])
            os.mkdir('./data/split_fr/' + str(document)[0:3])
            writer_NL.write(f"./data/split_nl/" + str(document)[0:3] + f"/NL_{document}")
            writer_FR.write(f"./data/split_fr/" + str(document)[0:3]+ f"/FR_{document}")
        except:
            writer_NL.write(f"./data/split_nl/" + str(document)[0:3] + f"/NL_{document}")
            writer_FR.write(f"./data/split_fr/" + str(document)[0:3]+ f"/FR_{document}")
        else:
            print(('document' + str(document) + 'not made'))
            pass         

In [ ]:
# 326
# 50.55,52.0,59
# 63.9,61.5,107